In [3]:
import pandas as pd
pd.set_option('precision', 3)

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import os

from neuralprophet import NeuralProphet

In [2]:
# !pip install neuralprophet

  Using cached convertdate-2.3.2-py3-none-any.whl (47 kB)
  Using cached holidays-0.11.3.1-py3-none-any.whl (155 kB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached torch_lr_finder-0.2.1-py3-none-any.whl (11 kB)
  Using cached PyMeeus-0.5.11.tar.gz (5.4 MB)
  Using cached hijri_converter-2.2.2-py3-none-any.whl (13 kB)
  Using cached korean_lunar_calendar-0.2.1-py3-none-any.whl (8.0 kB)
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=65fdd029a034d6d80ec0902544cdcf232113b66de9fa00fcdc60717746dde7b7
  Stored in directory: c:\users\selsh\appdata\local\pip\cache\wheels\33\3d\82\4579e9cca41ff991140b2e050bc6df3a38292f26e4fa06b15d
Successfully built pymeeus


In [4]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]

## Load Data From File

In [7]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is A450-BD8B

 Directory of C:\Users\selsh\repos\foodprice-forecasting

2021-12-07  01:28 PM    <DIR>          .
2021-12-07  01:28 PM    <DIR>          ..
2021-12-07  01:23 PM    <DIR>          .ipynb_checkpoints
2021-10-28  07:47 AM           387,352 36_48myforecasts_file.csv
2021-12-07  10:51 AM         1,606,969 all_data.csv.bak
2021-10-28  04:16 PM    <DIR>          Bakery and cereal products (excluding baby food)_CUSR0000SAF112nlags_24
2021-10-28  06:41 PM            11,580 compare_valid_results.ipynb
2021-12-07  10:51 AM             6,953 data.py
2021-12-07  10:51 AM    <DIR>          data_files
2021-10-28  03:32 PM             1,319 ensemble_models.csv
2021-12-07  10:51 AM               283 foodprice_categories.txt
2021-12-07  10:51 AM         1,497,344 forecasting-ensemble-experiment.ipynb
2021-12-07  10:51 AM           459,984 forecasting-ensemble-final.ipynb
2021-12-07  01:23 PM         1,059,462 forecasting-nbeats-experi

In [8]:
foodprice_categories = pd.read_csv("./foodprice_categories.txt", sep='\n', header=None)[0].to_list()
foodprice_df = pd.read_csv("./all_data.csv.bak", index_col=0)
foodprice_df = foodprice_df.set_index(pd.DatetimeIndex(foodprice_df.index))
foodprice_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations,DEXCAUS,...,XTIMVA01CAM657S,XTIMVA01CAM659S,XTIMVA01CAM664N,XTIMVA01CAM664S,XTIMVA01CAM667S,XTNTVA01CAM664N,XTNTVA01CAM664S,XTNTVA01CAM667S,TOTALNS,TOTALSL
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0,1.392,...,3.644,12.051,9.368e+09,9.496e+09,6.749e+09,6.906e+08,1.052e+09,7.479e+08,607.369,605.703
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4,1.392,...,1.965,16.745,9.495e+09,9.632e+09,6.881e+09,-9.880e+07,1.539e+08,1.099e+08,605.807,610.678
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2,1.392,...,-11.565,1.655,8.803e+09,8.529e+09,6.085e+09,9.138e+08,9.079e+08,6.478e+08,606.799,613.377
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1,1.392,...,13.334,10.821,1.034e+10,9.569e+09,6.897e+09,3.470e+08,6.563e+08,4.730e+08,614.367,619.658
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3,1.377,...,-4.236,6.160,9.598e+09,9.091e+09,6.605e+09,7.013e+08,6.893e+08,5.008e+08,621.915,625.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,157.7,145.3,146.2,163.9,156.8,144.5,176.7,142.2,153.4,1.203,...,-1.087,31.561,5.159e+10,4.979e+10,4.075e+10,3.661e+09,3.111e+09,2.546e+09,4259.604,4307.137
2021-07-01,157.9,146.4,146.6,165.2,157.6,141.7,180.9,141.9,154.8,1.240,...,1.990,18.580,5.106e+10,5.205e+10,4.156e+10,3.780e+08,1.129e+09,9.018e+08,4276.202,4323.730
2021-08-01,158.5,148.3,146.8,165.9,158.0,142.5,182.1,141.7,152.2,1.251,...,-2.312,14.723,5.245e+10,5.115e+10,4.060e+10,1.300e+09,2.674e+09,2.123e+09,4316.303,4337.489
2021-09-01,158.1,148.0,147.1,165.9,158.5,141.5,184.8,144.3,150.0,1.262,...,-3.461,8.557,5.080e+10,4.965e+10,3.920e+10,1.561e+09,2.277e+09,1.798e+09,4346.662,4367.402


## Experiment And Data Setup

In [9]:
report_sim_dates = pd.read_csv("./reportsimdates.txt", sep='\n', header=None)[0].to_list()
report_sim_dates

['2015-07-01',
 '2016-07-01',
 '2017-07-01',
 '2018-07-01',
 '2019-07-01',
 '2020-07-01']

In [10]:
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

In [13]:
dataset_df = foodprice_df[foodprice_categories].T
dataset_df

,1986-01-01,1986-02-01,1986-03-01,1986-04-01,1986-05-01,1986-06-01,1986-07-01,1986-08-01,1986-09-01,1986-10-01,...,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01
Bakery and cereal products (excluding baby food),69.3,70.3,70.6,71.3,71.2,71.1,71.7,71.9,71.7,71.1,...,154.2,157.1,156.8,156.2,157.8,157.7,157.9,158.5,158.1,159.3
Dairy products and eggs,70.9,70.8,71.1,71.0,71.4,71.1,71.3,71.5,71.8,71.8,...,141.6,143.1,144.9,146.1,146.6,145.3,146.4,148.3,148.0,146.6
"Fish, seafood and other marine products",60.6,61.3,61.3,61.4,61.9,62.0,62.2,62.7,63.1,63.6,...,143.4,143.9,144.9,145.1,147.6,146.2,146.6,146.8,147.1,149.3
Food purchased from restaurants,59.1,59.1,59.3,59.7,59.9,60.0,60.6,60.9,60.9,61.3,...,162.6,162.9,162.6,163.2,163.5,163.9,165.2,165.9,165.9,167.1
Food,67.3,66.9,67.0,67.7,68.2,68.4,69.2,69.5,69.9,70.2,...,155.0,155.6,155.5,155.4,156.6,156.8,157.6,158.0,158.5,159.2
"Fruit, fruit preparations and nuts",76.0,77.6,79.2,82.2,83.5,83.1,84.8,86.7,83.8,82.9,...,140.9,143.4,142.4,141.9,143.9,144.5,141.7,142.5,141.5,141.7
Meat,65.1,64.2,64.2,63.6,64.0,64.9,66.5,67.8,71.3,71.5,...,171.9,169.5,170.2,173.5,175.4,176.7,180.9,182.1,184.8,187.1
Other food products and non-alcoholic beverages,77.5,78.1,78.6,79.5,79.8,79.9,80.2,80.2,80.8,81.0,...,139.0,139.5,141.5,140.5,141.6,142.2,141.9,141.7,144.3,144.4
Vegetables and vegetable preparations,76.0,68.4,66.2,71.1,75.3,74.1,75.7,71.9,66.6,70.7,...,162.5,163.8,157.4,151.1,153.8,153.4,154.8,152.2,150.0,149.4


In [33]:
data_sources = ["DEXCAUS",
                "DCOILWTICO",
                "WILL5000IND",
                "VXOCLS",
                "CUSR0000SAF112",
                "CUSR0000SAF113",
                "CPIFABSL",
                "UNRATE",
                "FEDFUNDS",
                "IRLTLT01CAM156N",
                "LRUNTTTTCAM156S",
                "CPALCY01CAM661N",
                "CPGRLE01CAM657N",
                "QCAR368BIS"
               ]

In [32]:
print(foodprice_df)

            Bakery and cereal products (excluding baby food)  \
1986-01-01                                              69.3   
1986-02-01                                              70.3   
1986-03-01                                              70.6   
1986-04-01                                              71.3   
1986-05-01                                              71.2   
...                                                      ...   
2021-06-01                                             157.7   
2021-07-01                                             157.9   
2021-08-01                                             158.5   
2021-09-01                                             158.1   
2021-10-01                                             159.3   

            Dairy products and eggs  Fish, seafood and other marine products  \
1986-01-01                     70.9                                     60.6   
1986-02-01                     70.8                                    

In [34]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}
def get_prophet_df(foodprice_df, food_category, dates):
    df = foodprice_df[food_category][dates]
    df = df.reset_index()
    df = df.rename({'REF_DATE':'ds', 'index':'ds', food_category:'y'}, axis=1)
    return df

def get_neuralprophet_df(foodprice_df, food_category, dates, lagged_regressor):
    prophet_df = pd.DataFrame({'ds': foodprice_df[food_category][dates].index, 
                           'y':foodprice_df[food_category].loc[foodprice_df.index][dates], 
                           lagged_regressor: foodprice_df[lagged_regressor].loc[foodprice_df.index][dates],
                          })
    prophet_df.reset_index(drop=True, inplace=True)

    return prophet_df
    

# Hyperparameter tuning using ParameterGrid

In [39]:
from sklearn.model_selection import ParameterGrid
params_grid = {'n_forecasts':[18],
              'n_lags':[36]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1
print('Total Possible Models',cnt)
#               'n_lags':[24, 36, 48, 60]}

Total Possible Models 1


## Fit Models and Save Forecasts

Fit a model using data up to the `report_sim_date` and use it to produce forecasts for the following 18 months. Collect evaluation metrics for comparion and save the forecasts for use in the ensembling experiments.

In [54]:
# all_foodprice_columns 
print(foodprice_df.columns[:9])


Index(['Bakery and cereal products (excluding baby food)',
       'Dairy products and eggs', 'Fish, seafood and other marine products',
       'Food purchased from restaurants', 'Food',
       'Fruit, fruit preparations and nuts', 'Meat',
       'Other food products and non-alcoholic beverages',
       'Vegetables and vegetable preparations'],
      dtype='object')


In [46]:
%%time
## takes 13 min for one pair of var and p
all_output_paths = []

## we can have these two outside the loop since the key will be based on the output_path and food_category
all_valid_metrics = {}
all_forecasts = {}
data_sources = ["DEXCAUS"]
for var in data_sources:
    for p in grid:
        details = str(var) + '_nlags_' + str(p['n_lags'])
        output_path = './output/neuralprophet_202110_' + details
        if not os.path.exists(output_path):
            print("making new directory for: ", output_path)
            os.mkdir(output_path)
        else:
            print("already done so skipping")
            continue
        all_output_paths.append(output_path)
        
        
        for food_category in foodprice_df.columns[:9]:
            
            annual_valid_metrics = {}
            annual_forecasts = {}
                        
            for date in report_sim_dates:
                
                train_dates = sim_train_dates[date]
                valid_dates = sim_valid_dates[date]
                
                
                train_model =NeuralProphet(n_forecasts = p['n_forecasts'],                          #initialize model
                                           n_lags = p['n_lags'])
                train_df = get_neuralprophet_df(foodprice_df, food_category, train_dates, str(var)) #select data
                train_model = train_model.add_lagged_regressor(names=[str(var)])                   #lagged regressor to model
                train_model.fit(train_df, freq='MS')                                              #fit the model
                
                
                future = train_model.make_future_dataframe(train_df, periods=18)                #create the future period
                forecast = train_model.predict(future)                                        #produce the future period forecast
                
                
                all_yhats = []
                for yhat in ["yhat" + str(i) for i in range(1,19)]:
                    all_yhats.append((forecast.loc[forecast[yhat].first_valid_index()][yhat]))  #get all yhats from forecast
                print("this is all yhats ", all_yhats)

                
                valid_df = get_neuralprophet_df(foodprice_df, food_category, valid_dates, str(var))
                
                ##special case for 2020 forecast
                if len(valid_df) == 14:
                    print("case of 2020 forecast")
                    all_yhats = all_yhats[:-4]
                    
                ##special case for 2020 forecast
                if len(valid_df) == 15:
                    print("IN A NEW SPECIAL CASE")
                    all_yhats = all_yhats[:-3]
                
                ## compute valid metrics
                valid_metrics = {metric_name: metric_fn(y_true=valid_df.y, y_pred=all_yhats) for metric_name, metric_fn in metrics.items()}
                valid_metrics_series = pd.Series(valid_metrics)
                
                annual_valid_metrics[date] = pd.DataFrame(valid_metrics_series).mean(axis=1)
                annual_forecasts[date] = pd.Series(all_yhats)
                            
            
                break ### loop for dates 
                
            all_valid_metrics[(output_path, food_category)] = annual_valid_metrics
            all_forecasts[(output_path, food_category)] = annual_forecasts
                
            print("this is all_valid_metrics", all_valid_metrics)
            mean_valid_metrics = {}
            for (output_path,food_category), valid_scores in all_valid_metrics.items():
                mean_valid_metrics[food_category] = pd.DataFrame(valid_scores).mean(axis=1)

            mean_valid_metrics_df = pd.DataFrame(mean_valid_metrics).T
            mean_valid_metrics_df.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
            
        
            # break ## loop for food category
        
        break ## loop for n_lags

    break ##loop for lagged regressors



INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 233


  0%|          | 0/224 [00:00<?, ?it/s]

making new directory for:  ./output/neuralprophet_202110_DEXCAUS_nlags_36


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.38E-02, min: 3.45E-01


  0%|          | 0/224 [00:00<?, ?it/s]

...

  0%|          | 0/224 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.79E-03, min: 4.16E-01


  0%|          | 0/224 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.79E-03, min: 5.49E-01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 5.79E-03
Epoch[110/233]:  47%|█▉  | 110/233 [00:07<00:08, 14.61it/s, SmoothL1Loss=0.00233, MAE=1.88e+5, RMSE=2.45e+5, RegLoss=0]


KeyboardInterrupt: 

## Save validation forecasts to file (for ensembling)


In [38]:
date_forecasts = {}
for date in report_sim_dates:
    valid_dates = sim_valid_dates[date]
    food_forecasts = {}
    for my_output_path in all_output_paths:
        for food_category in food_categories:
            food_forecasts[food_category] = all_forecasts[(my_output_path,food_category)][date]
        date_forecasts[date] = pd.DataFrame(food_forecasts)
        date_forecasts[date] = date_forecasts[date].set_index(pd.DatetimeIndex(valid_dates))
        date_forecasts[date].to_csv(f"{my_output_path}/forecasts_{date}.csv")

KeyError: ('./output/neuralprophet_202110_DEXCAUS_nlags_36', 'Bakery and cereal products (excluding baby food)')